---
# Projeto 2 - Ciência dos Dados

Nome: Daniel Pucciariello

Nome: Stephanie Liu

---

___
# Classificador automático de sentimento
## Preparando o ambiente no jupyter:

In [2]:
%%capture

#Instalando o tweepy
!pip install tweepy

In [3]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle

In [4]:
import re 

def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    import string
    punctuation = '[!-.:?;/]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    
    text_subbed = re.sub(pattern, ' ', text)
    text_subbed=text_subbed.replace("\n", " ")
    text_subbed=text_subbed.replace("…", " ")
    text_subbed=text_subbed.replace("@", " ")
    text_subbed=text_subbed.replace("rt", "")
    text_subbed=text_subbed.replace("https", "")
    
    return text_subbed
#tirar coisas com https

In [5]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @fulano

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
### Montando o Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

---

In [6]:
data_treinamento = pd.read_excel("Excel_pastel.xlsx",sheet_name="Treinamento")

In [7]:
irrelevantes_raw = " ".join(data_treinamento[data_treinamento.Classe<=1].Treinamento)

In [8]:
irrelevantes = cleanup(irrelevantes_raw)
#irrelevantes

In [9]:
tabela_relativa_ngostam_irrelevantes = pd.Series(irrelevantes.lower().split()).value_counts(True)
tabela_absoluta_ngostam_irrelevantes = pd.Series(irrelevantes.lower().split()).value_counts(False)


In [10]:
tabela_relativa_gostam_raw = " ".join(data_treinamento[data_treinamento.Classe==2].Treinamento)

In [11]:
tabela_relativa_gostam = pd.Series(tabela_relativa_gostam_raw.lower().split()).value_counts(True)
tabela_absoluta_gostam = pd.Series(tabela_relativa_gostam_raw.lower().split()).value_counts(False)

In [12]:
Multiplicando_tabela_naogostam = tabela_relativa_ngostam_irrelevantes*2

In [13]:
tudo=irrelevantes_raw + tabela_relativa_gostam_raw
tudo=cleanup(tudo)
tudo_absoluto=pd.Series(tudo.lower().split()).value_counts(False)

In [14]:
data_teste = pd.read_excel("Excel_pastel.xlsx",sheet_name="Teste")

In [15]:
#data_teste_idx= data_teste.set_index("Teste")
lista=[]
lista2=[]
for i in data_teste.Teste:
    gosta=((tabela_absoluta_gostam[i.split()]+1)/(len(tabela_absoluta_gostam)+len(tudo_absoluto))).prod()
    ngosta=((tabela_absoluta_ngostam_irrelevantes[i.split()]+1)/(len(tabela_absoluta_ngostam_irrelevantes)+len(tudo_absoluto))).prod()
    if gosta>ngosta:
        valor=2
    elif gosta<ngosta:
        valor=1

    lista.append(valor)
data_teste.Classe = lista
data_teste

C:\Users\danie\Anaconda3\lib\site-packages\pandas\core\series.py:951: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self.loc[key]


,Teste,Classe
0,"@podecrematheus salva esse pastel amigo, eu fa...",1
1,daiso me de todos os seus giz pastel,2
2,"@rodrigobocardi feira vem na memória ""afetiva ...",2
3,comi dois pastel com caldo de cana 😋🤤,2
4,saudade pastel de camarão &gt;&lt;,2
5,"eu tinha 2 reais,eu podia ter comprado papel a...",1
6,vontade de comer pastel,2
7,se fosse o bestao aqui deixaria um pastel p to...,1
8,comendo pastel 7 da manha dps reclama q ta c g...,2
9,"@taegifix roxo, preto e azul pastel",2
